<font size="4">
    <h1>
        Chapter 1- Migrating, consolidating to Parquet and first analysis
    </h1>

In [1]:
# Just take all width for viz
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

import os

In [2]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config("spark.driver.memory", "10g") \
    .appName("steam-analysis-eda") \
    .getOrCreate()

In [3]:
dataset_path = os.path.join(os.path.dirname(os.path.realpath("")), "data/steam-dataset/")

games_df = spark.read.parquet('file://' + dataset_path + "steam_analysis.App_ID_Info")

games_df.printSchema()

root
 |-- appid: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Required_Age: string (nullable = true)
 |-- Is_Multiplayer: string (nullable = true)



- Checking dataset

In [4]:
games_df \
    .filter(F.col('Type') == "game") \
    .show(truncate = False)

+------+----------------------------------------------+----+-----+-------------------+------+------------+--------------+
|appid |Title                                         |Type|Price|Release_Date       |Rating|Required_Age|Is_Multiplayer|
+------+----------------------------------------------+----+-----+-------------------+------+------------+--------------+
|414120|Modbox                                        |game|14.99|2016-04-05 00:00:00|-1    |0           |0             |
|363020|IPackThat                                     |game|64.99|2015-10-06 00:00:00|-1    |0           |0             |
|374050|Infinium Strike                               |game|0    |2016-07-14 00:00:00|-1    |0           |0             |
|306410|Crystals of Time                              |game|4.99 |2014-06-13 00:00:00|-1    |0           |0             |
|363050|Let's Explore the Airport (Junior Field Trips)|game|6.99 |2015-04-24 00:00:00|-1    |0           |0             |
|375770|Quantum Conscien

- Working on single licence

In [5]:
games_df \
    .filter(F.col("Title").contains('Call of Duty')) \
    .filter(F.col('Type') == "game") \
    .show(truncate = False)

+------+------------------------------------------------+----+-----+-------------------+------+------------+--------------+
|appid |Title                                           |Type|Price|Release_Date       |Rating|Required_Age|Is_Multiplayer|
+------+------------------------------------------------+----+-----+-------------------+------+------------+--------------+
|115300|Call of Duty®: Modern Warfare® 3                |game|39.99|2011-11-08 00:00:00|-1    |17          |1             |
|388520|Call of Duty®: Black Ops III                    |game|59.99|2015-11-05 00:00:00|-1    |17          |1             |
|42690 |Call of Duty®: Modern Warfare® 3                |game|39.99|2011-11-08 00:00:00|-1    |17          |1             |
|10190 |Call of Duty®: Modern Warfare® 2                |game|19.99|2009-11-11 00:00:00|86    |0           |1             |
|42680 |Call of Duty®: Modern Warfare® 3                |game|39.99|2011-11-08 00:00:00|-1    |17          |1             |
|2630  |

- Before joining, check that schema are the compatible

In [6]:
# Add Old dataset
games_old_df = spark.read.parquet('file://' + dataset_path + "steam_analysis.App_ID_Info_Old")

games_old_df.printSchema()

root
 |-- appid: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Required_Age: string (nullable = true)
 |-- Is_Multiplayer: string (nullable = true)



In [7]:
games_old_df \
    .filter(F.col('Type') == "game") \
    .show(truncate = False)

+-----+------------------------------+----+-----+-------------------+------+------------+--------------+
|appid|Title                         |Type|Price|Release_Date       |Rating|Required_Age|Is_Multiplayer|
+-----+------------------------------+----+-----+-------------------+------+------------+--------------+
|10   |Counter-Strike                |game|9.99 |2000-11-01 00:00:00|-1    |0           |1             |
|20   |Team Fortress Classic         |game|4.99 |1999-04-01 00:00:00|-1    |0           |1             |
|30   |Day of Defeat                 |game|4.99 |2003-05-01 00:00:00|79    |0           |1             |
|40   |Deathmatch Classic            |game|4.99 |2001-06-01 00:00:00|-1    |0           |1             |
|50   |Half-Life: Opposing Force     |game|4.99 |1999-11-01 00:00:00|-1    |0           |1             |
|60   |Ricochet                      |game|4.99 |2000-11-01 00:00:00|-1    |0           |1             |
|70   |Half-Life                     |game|9.99 |1998-1

- For sales data collection using VGChartz in later article : extract distinct game names in dataset

In [8]:
# Get all distinct game names, from old and current dataset

distinct_games_df = games_df \
    .union(games_old_df) \
    .select('Title') \
    .distinct()

distinct_games_df.show()

output_dir = os.path.join(os.path.dirname(os.path.realpath("")), "data/extracts/")

filepath = os.path.join(output_dir, "steam-dataset-distinct-names")
distinct_games_df \
    .write \
    .mode("overwrite") \
    .csv("file://" + filepath)

+--------------------+
|               Title|
+--------------------+
|Call of Duty®: Mo...|
|Theatre Of The Ab...|
|Crusader Kings II...|
|          Dungeons 2|
|            FreezeME|
|SHOGUN: Total War...|
|Zombie Training S...|
|  The Black Watchmen|
|East Tower - Kurenai|
|Back to the Futur...|
|Dota 2 - The Inte...|
|Legacy of Kain: S...|
|Fantasy Grounds -...|
|The Cat Machine -...|
|         Inexistence|
|Saints Row IV - H...|
| Stick 'Em Up 2 Demo|
|NEON STRUCT Sound...|
|      AirMech® Prime|
|Magic 2012 Foil C...|
+--------------------+
only showing top 20 rows



#### 2. Start consolidation on games

In [17]:
# Get larger dataset

files = [
    "steam_analysis.Achievements_Percentages",
    "steam_analysis.Games_1",
    "steam_analysis.Games_2",
    "steam_analysis.Games_Daily"
]

for filename in files:
    print('Filename :', filename)
    df = spark.read.parquet("file://" + dataset_path + filename, header=True)
    df.printSchema()

del df

Filename : steam_analysis.Achievements_Percentages
root
 |-- Appid: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Percentage: string (nullable = true)

Filename : steam_analysis.Games_1
root
 |-- steamid: decimal(20,0) (nullable = true)
 |-- appid: long (nullable = true)
 |-- playtime_2weeks: long (nullable = true)
 |-- playtime_forever: long (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)

Filename : steam_analysis.Games_2
root
 |-- steamid: decimal(20,0) (nullable = true)
 |-- appid: long (nullable = true)
 |-- playtime_2weeks: long (nullable = true)
 |-- playtime_forever: long (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)

Filename : steam_analysis.Games_Daily
root
 |-- steamid: decimal(20,0) (nullable = true)
 |-- appid: long (nullable = true)
 |-- playtime_2weeks: long (nullable = true)
 |-- playtime_forever: long (nullable = true)
 |-- dateretrieved: timestamp (nullable = true)



#### Retreive most played games

- Les jeux de Valve doivent être exclus de l'analyse
    - Exemple : le sorting par temps de jeu "forever"

#### Adding Playtime

In [10]:
playtime_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_1")

# sort: Provoke "java.lang.OutOfMemoryError: Java heap space" error if java memory is set to default
games_df \
    .filter(F.col('Type') == 'game') \
    .join(playtime_df, on = 'appid') \
    .dropna() \
    .select(*games_df.columns, "playtime_2weeks" ,'playtime_forever') \
    .sort(F.col('playtime_2weeks').desc()) \
    .show(truncate = False)

AnalysisException: Path does not exist: file:/home/neadex/steam-analysis/data/extracts/steam_analysis.Games_1;

- If sorting by playtime_forever, almost only Valve games : could normalize playtime_forever by time elapsed since first record

In [9]:
games_df \
    .filter(F.col('Type') == 'game') \
    .join(playtime_df, on = 'appid') \
    .dropna() \
    .select(*games_df.columns, "playtime_2weeks" ,'playtime_forever') \
    .sort(F.col('playtime_forever').desc()) \
    .show(truncate = False)

NameError: name 'playtime_df' is not defined

#### Adding Developer and filtering Valve product

- Valve products needs to be filtered because too much data on them (recorded since Steam start or few time later)

In [199]:
dataset_path

'/home/neadex/steam-analysis/data/extracts/'

In [198]:
dev_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Developers", header = True)
dev_old_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Developers_Old", header = "True")

dev_df = dev_df.union(dev_old_df)

# Remove headers added as rows. Need to investigate on behavior origin.
dev_df = dev_df \
    .filter((F.col('Developer') == 'Developer') == False)

games_df \
    .filter(F.col('Type') == 'game') \
    .join(dev_df, on = 'appid') \
    .filter(F.col('Developer') != "Valve") \
    .dropDuplicates() \
    .write \
    .mode("overwrite") \
    .parquet("file://" + output_dir + "steam-dataset_games_28-12")

AnalysisException: Path does not exist: file:/home/neadex/steam-analysis/data/extracts/steam_analysis.Games_Developers;

#### Adding Publisher

In [ ]:
pub_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Publishers", header = True)
pub_old_df = spark.read .parquet("file://" + dataset_path + "steam_analysis.Games_Publishers_Old", header = "True")

pub_df = pub_df.union(pub_old_df)

# Filter Valve
pub_df = pub_df.filter(F.col('Publisher') != 'Valve')

# Load dataset from last step
df = spark.read.parquet("file://" + output_dir + "steam-dataset_games_28-12")

df = df.join(pub_df, on = 'appid')

df \
    .dropDuplicates() \
    .write \
    .mode("overwrite") \
    .parquet("file://" + output_path + "steam-dataset_games_28-12_2")

#### Adding achievements

- Flattening structure: need aggregation on achievements

In [185]:
ach_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Achievements_Percentages", header=True)

# Rename column to allow join
# filter headers set as columns
ach_df = ach_df \
    .withColumnRenamed('Appid', 'appid') \
    .join(pub_df, on = 'appid') \
    .filter(F.col('appid') != 'appid') \
    .groupBy('appid') \
    .agg(F.mean('Percentage').alias('average_achievements_percentages'))

# Load dataset from last step
df = spark.read.parquet("file://" + output_dir + "steam-dataset_games_28-12_2")

df = df.join(ach_df, on = "appid")

df \
    .dropDuplicates() \
    .write \
    .mode("overwrite") \
    .parquet("file://" + output_path + "steam-dataset_games_28-12_3")

NameError: name 'pub_df' is not defined

In [ ]:
df.printSchema()

### Games Daily : player usage

- Game playing data for a select subset of users. Each user's data in the subset was requested repeatedly, every day for five days.

In [ ]:
df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Daily")

df.printSchema()

df = df \
    .groupBy('steamid') \
    .agg(F.collect_list('appid').alias('owned_appid')) \
    .withColumn('n_owned', F.size(F.col('owned_appid'))) \
    .sort(F.col('n_owned'))

df.printSchema()

n_owned_median = df \
    .approxQuantile('n_owned', [0.5], 0.01)[0]

print('Number of owned game median :', n_owned_median)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = df.toPandas().n_owned,
        name = "N Owned",
        histnorm='probability'
    )
)

fig.update_layout(title = 'Number of owned game histogram')

fig.show()

In [ ]:
# Filter extreme n_owned values

max_owned = 500

filtered_df = df \
    .filter(F.col('n_owned') <= max_owned) \
    .sort(F.col('n_owned'))

# Display filtered results
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = filtered_df.toPandas().n_owned,
        name = "N Owned"
    )
)

fig.update_layout(title = 'Number of owned game histogram')

fig.show()

#### Notes on n_owned :

- Presqu'aucune occurence pour les tranches 0-4, 5-9 et 10-14 ? Etrange.
- Forme générale de l'histogramme plutôt irrégulière. Etonnant pour un dataset aussi large. Les histo sur des volumes tels sont le plus souvent très lisses.
- Need to remove non-game appid and Valve appid using Developer/Publisher dataset

In [46]:
df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Daily")

# Using previously cleaned dataset, without any Valve appid to implicitly remove appid with inner join
# Previous dataset also filter non-game appid
publishers = spark.sql(
    'select appid, Publisher, Developer from parquet.`{}`'.format("file://" + output_path + "steam-dataset_games_28-12_3")
)

df = df \
    .join(publishers, on = 'appid') \
    .dropna()

df = df \
    .groupBy('steamid') \
    .agg(F.collect_set('appid').alias('owned_appid')) \
    .withColumn('n_owned', F.size(F.col('owned_appid'))) \
    .sort(F.col('n_owned').desc())

df.show()

+-----------------+--------------------+-------+
|          steamid|         owned_appid|n_owned|
+-----------------+--------------------+-------+
|76561198004881524|[277870, 253030, ...|    143|
|76561197997403214|[264340, 266010, ...|    142|
|76561198001578025|[39150, 253030, 2...|    111|
|76561197981390154|[34330, 24240, 24...|    101|
|76561198052529626|[305260, 8930, 21...|    101|
|76561197970497684|[223470, 304650, ...|     97|
|76561197986793562|[223470, 8930, 25...|     97|
|76561198033149350|[221640, 8930, 23...|     94|
|76561197999737253|[221640, 292120, ...|     74|
|76561198037122955|[209170, 35450, 1...|     63|
|76561198031324442|[277870, 24240, 2...|     61|
|76561198011207815|[264340, 275490, ...|     57|
|76561198010825215|[231020, 266010, ...|     57|
|76561198039474824|[264340, 24240, 2...|     49|
|76561198010323350|[8930, 34330, 230...|     49|
|76561198014231534|[223470, 8930, 21...|     49|
|76561197997258359|[218620, 290320, ...|     47|
|76561197996524744|[

In [191]:
import plotly.graph_objects as go

# Filter extreme n_owned values
max_owned = 500

filtered_df = df \
    .filter(F.col('n_owned') <= max_owned) \
    .sort(F.col('n_owned'))

# Get some describe statistics
filtered_df.describe('n_owned').show()

# Display filtered results
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = filtered_df.toPandas().n_owned,
        name = "N Owned"
    )
)

fig.update_layout(
    title = 'Number of owned game histogram',
    template = 'plotly_dark'
)

fig.show()

+-------+------------------+
|summary|           n_owned|
+-------+------------------+
|  count|             36432|
|   mean|3.3256203337725077|
| stddev|3.9212561840849087|
|    min|                 1|
|    max|               143|
+-------+------------------+



- Looking at genre combinations
- We can expect to find common mixed genres by cooccurrence

- Load Genres dataset and get all distinct owned genre by player

In [47]:
genre_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Genres")
genre_old_df = spark.read.parquet("file://" + dataset_path + "steam_analysis.Games_Genres_Old")

genre_df = genre_df.union(genre_old_df)

df = df \
    .select('steamid', F.explode('owned_appid').alias('appid')) \
    .join(games_df, on = 'appid') \
    .join(genre_df, on = 'appid') \
    .groupBy('steamid') \
    .agg(F.collect_set('Genre').alias('game_genres')) \
    .join(df, on = 'steamid')

df.show()

+-----------------+--------------------+--------------------+-------+
|          steamid|         game_genres|         owned_appid|n_owned|
+-----------------+--------------------+--------------------+-------+
|76561197963162410|[RPG, Massively M...|[251730, 248820, ...|      7|
|76561197965118034|[RPG, Massively M...|[224460, 8930, 21...|     11|
|76561197967145068|[RPG, Action, Ind...|[248820, 207170, ...|      6|
|76561197968164238|[RPG, Action, Ind...|    [113200, 250900]|      2|
|76561197970363728|[RPG, Simulation,...|    [107410, 290080]|      2|
|76561197970403961|  [Action, Strategy]|      [49540, 40100]|      2|
|76561197970406546|               [RPG]|             [72850]|      1|
|76561197970507908|[Action, Indie, A...|[253030, 26800, 2...|      3|
|76561197970569105|[RPG, Sports, Sim...|[241930, 49520, 2...|      7|
|76561197970750747|    [Sports, Action]|[296770, 209650, ...|      3|
|76561197972092412|[RPG, Massively M...|     [293540, 72850]|      2|
|76561197972209171| 

- MapReduce :
    - Combine owned genres by player and Map stage : create dict {"Genre1-Genre2" : 1}
    - Reduce by key with sum aggregation

- Create UDF to combine genres and count

    **NOTES** : Les jeux possédés sont crés par un SET, le count n'est pas nécessaire. Il suffit de faire correspondre "1" pour chaque key.
        Amélioration : compter les multiples occurences d'un même genre hybride.
                       example : Si 2 RPG-Action possédés => {"RPG-Action" : 2}
                           steps : 
                               - Remplacer "collect_set" par "collect_list"
                               - Récupérer pour chaque joueur les combinaisons distinctes
                               - Pour chaque combinaison, compter le nombre d'occurence

In [84]:
from itertools import combinations
from collections import Counter

def comb_count(x):
    count = Counter(list(combinations(x, 2)))
    count = dict(zip(["-".join(x) for x in count.keys()], count.values()))
    return count

example = ['RPG', 'Massively Multiplayer', 'Action', 'Indie', 'Adventure']
comb_count(example)

{'RPG-Massively Multiplayer': 1,
 'RPG-Action': 1,
 'RPG-Indie': 1,
 'RPG-Adventure': 1,
 'Massively Multiplayer-Action': 1,
 'Massively Multiplayer-Indie': 1,
 'Massively Multiplayer-Adventure': 1,
 'Action-Indie': 1,
 'Action-Adventure': 1,
 'Indie-Adventure': 1}

In [106]:
count_udf = F.udf(comb_count, MapType(StringType(), IntegerType()))

mixed_genres_counts = df \
    .withColumn('cooccurrences', count_udf(df['game_genres'])) \
    .select(F.explode('cooccurrences')) \
    .groupBy('key') \
    .agg(F.sum('value').alias('mixed_genres_sum')) \
    .sort(F.col('mixed_genres_sum').desc()) \

mixed_genres_counts.show(truncate = False)

+-------------------+----------------+
|key                |mixed_genres_sum|
+-------------------+----------------+
|RPG-Action         |16491           |
|Action-Indie       |14732           |
|Action-Adventure   |14097           |
|Action-Strategy    |11735           |
|RPG-Indie          |11659           |
|Indie-Adventure    |11544           |
|RPG-Adventure      |11041           |
|Indie-Strategy     |9795            |
|RPG-Strategy       |9763            |
|Action-Free to Play|9051            |
|Simulation-Action  |7939            |
|Simulation-Strategy|7881            |
|Adventure-Strategy |7795            |
|Indie-Early Access |7180            |
|Simulation-Indie   |6949            |
|Action-Early Access|6621            |
|Indie-Free to Play |6601            |
|Action-Casual      |6193            |
|RPG-Simulation     |6150            |
|RPG-Free to Play   |6032            |
+-------------------+----------------+
only showing top 20 rows



In [120]:
mixed_genres_stats = mixed_genres_counts.select('mixed_genres_sum').describe()

mixed_genres_stats.show()

+-------+------------------+
|summary|  mixed_genres_sum|
+-------+------------------+
|  count|               206|
|   mean| 1553.495145631068|
| stddev|3083.8836027947104|
|    min|                 1|
|    max|             16491|
+-------+------------------+



In [183]:
# Display filtered results
fig = go.Figure()

pd_df = mixed_genres_counts \
    .toPandas()

fig.add_trace(
    go.Bar(
        x = pd_df.key,
        y = pd_df.mixed_genres_sum,
        name = "Mixed genres counts"
    )
)

fig.update_layout(
    title = 'Mixed genre games histogram',
    template = 'plotly_dark'
)
fig.update_xaxes(tickangle = 45)

fig.show()

- Nettoyage du dataset pour une meilleure lisibilité directe
    - Plutôt que d'utiliser un seuil en valeur absolue, nous allons calculer cette valeur à partir du dataset pour garantir la compatibilité du seuil avec le dataset, et assurer la généralisation de la démarche. Si le dataset change, la valeur accompagne les évolutions.
    - Distribution très asymétrique : Utilisation de la médiane

In [178]:
# Set percentile threshold
pop_threshold = 0.5

threshold = mixed_genres_counts \
    .approxQuantile('mixed_genres_sum', [pop_threshold], 0.01)[0]
    
print('Threshold:', threshold)

Threshold: 37.0


In [184]:
# Display filtered results
fig = go.Figure()

pd_df = mixed_genres_counts \
    .filter(F.col('mixed_genres_sum') > threshold) \
    .toPandas()

fig.add_trace(
    go.Bar(
        x = pd_df.key,
        y = pd_df.mixed_genres_sum,
        name = "Mixed genres counts"
    )
)

fig.update_layout(
    title = 'Mixed genre games counts',
    template = 'plotly_dark'
)

fig.update_xaxes(tickangle = 45)

fig.show()

- distance entre RPG-Massively Multiplayer / Massively Multiplayer - Early Access
- Trouver quel genre comporte le plus d'early access
    - prédire l'évolution de l'early access par genre en fonction du temps